네이버 시가총액 (코스피500개, 코스닥300개)

In [1]:
import csv
import requests
import datetime
from bs4 import BeautifulSoup

def stock_data(columns, sosok):
    data = list()
    data = data + [sosok]
    code = list()    
    #fn_link = list()
    for column in columns:        
        data = data + [column.text.strip()]            
        a = column.find("a",{"class":"tltle"})

        if a:
            code = [a.attrs["href"][20:]]
            #code = ["https://finance.naver.com" + a.attrs["href"]] 
            code_str = str(code)[4:10]
            #fn_link = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A"+code_str+"&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701"
            fn_link = "https://comp.fnguide.com/SVO/WooriRenewal/Snapshot.asp?pGB=12&gicode=A"+code_str
            #print(fn_link)
            data = data + code + [fn_link]

    return data[0:-1]



today = datetime.date.today().strftime('%y%m%d')
filename = today + "_sise_code.csv"

f = open(filename, "w", encoding="euc-kr", newline="")
writer = csv.writer(f)

title = "구분,No,종목명,종목코드,fn링크,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE".split(",")
print(type(title))
writer.writerow(title)

for sosok in range(2):
    if sosok==0:
        last_page = 11
    else:
        last_page = 7    

    url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=" + str(sosok) + "&page="

    for page in range(1, last_page):
        res =requests.get(url + str(page))
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "lxml")

        data_rows = soup.find("table", attrs={"class":"type_2"}).find("tbody").find_all("tr")
        for row in data_rows:
            columns = row.find_all("td")
            #print(columns)
            #print("*****")
            if len(columns) <= 1:
                continue        
            #data = [column.get_text().strip() for column in columns]
            #print(data)
            #writer.writerow(data)
            
            data = stock_data(columns, sosok)
            #print(type(data))
            #print(data)

            writer.writerow(data)

print('save')
f.close()


<class 'list'>
save


관심종목 등락률 조회 / 관심종목에 종목코드 값 추가

In [10]:
import csv
import pandas as pd
import re
import datetime

pd.set_option('display.max_rows', None)

today = datetime.date.today().strftime('%y%m%d')
filename = today + "_sise_code.csv"

df = pd.read_csv(filename, encoding="euc-kr")
df_interest = pd.read_csv("interest.csv", encoding="euc-kr")

df['등락률2']=df['등락률'].str.replace('%','').astype(float)
df['전일비2']=df['전일비'].str.replace(r'\s+',' ', regex=True)

df_stock = df.loc[:, ['구분','종목명','종목코드','현재가','전일비2','등락률2','외국인비율','PER','ROE']]
df_stock_updown_sort = df_stock.sort_values(by=['등락률2','현재가'], ascending=True)
# df_stock_siga_sort = df_stock.sort_values(by=['시가총액'])

#df_pr_ch = df3.loc[((df3['등락률2'] > 10) | (df3['등락률2'] < -5)) & (df3['외국인비율'] > 5), ['구분','종목명','현재가','전일비2','등락률2','외국인비율','PER','ROE']]
#display(df_pr_ch)   

interest_list = df_interest['종목명'].tolist()
df_sname =df_stock.loc[df_stock['종목명'].isin(interest_list)]                                                                                                                                                                            
display(df_sname)

f = open("interest.csv", "w", encoding="euc-kr", newline="")
df_interest_update = df_sname.loc[:,['종목명','종목코드']]
df_interest_update.to_csv(f)

f.close()

print('관심종목 종목코드 입력완료')



,구분,종목명,종목코드,현재가,전일비2,등락률2,외국인비율,PER,ROE
0,0,삼성전자,e=005930,"109,300",상승 900,0.83,52.28,22.70,9.03
1,0,SK하이닉스,e=000660,"541,000","하락 3,000",-0.55,53.14,11.03,31.06
4,0,삼성전자우,e=005935,"80,300",상승 200,0.25,76.93,16.67,NaN
14,0,NAVER,e=035420,"248,500","하락 1,000",-0.40,38.78,17.95,7.90
24,0,LG화학,e=051910,"385,500","상승 4,000",1.05,34.49,-20.42,-2.11
27,0,삼성SDI,e=006400,"312,500","상승 5,000",1.63,24.31,-42.81,3.13
35,0,삼성전기,e=009150,"259,000","하락 4,000",-1.52,38.12,29.06,8.16
42,0,LG전자,e=066570,"96,600","하락 3,100",-3.11,32.25,16.25,1.81
53,0,SK텔레콤,e=017670,"53,900",하락 500,-0.92,36.23,19.73,10.83
56,0,한미반도체,e=042700,"117,800","하락 1,300",-1.09,6.73,46.91,27.43


관심종목 종목코드 입력완료


관심종목 실시간 조회

In [11]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

def naver_data(code):
    url = "https://finance.naver.com/item/main.nhn?code=" + code    
    res =requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")

    try :
        s_name = soup.find('div',{'class':'h_company'}).find('h2').text.replace(u'\xa0','')
        s_price = soup.find('p',{'class':'no_today'}).find('span',{'class':'blind'}).text.replace(u'\xa0','')
        s_ch = soup.find('p',{'class':'no_exday'}).find_all('em')[0].find('span',{'class':'blind'}).text.replace(u'\xa0','')
        s_ch1 = soup.find('p',{'class':'no_exday'}).find_all('em')[1].find('span',{'class':'ico'}).text.replace(u'\xa0','')
        s_ch2 = soup.find('p',{'class':'no_exday'}).find_all('em')[1].find('span',{'class':'blind'}).text.replace(u'\xa0','')
        #s_pr1 = soup.find('dl',{'class':'blind'}).find_all('dd')[4].text.replace(u'\xa0','')
        #s_pr2 = soup.find('dl',{'class':'blind'}).find_all('dd')[5].text.replace(u'\xa0','')
        s_pr3 = soup.find('dl',{'class':'blind'}).find_all('dd')[6].text.replace(u'\xa0','')[3:]
        s_pr4 = soup.find('dl',{'class':'blind'}).find_all('dd')[8].text.replace(u'\xa0','')[3:]
        s_siga = soup.find('div',{'class':'first'}).find('table').find_all('tr')[0].find('td').find('em').text.strip().replace(u'\t','').replace(u'\n','')        
        s_fore = soup.find('div',{'class':'gray'}).find('table').find_all('tr')[2].find('td').find('em').text.strip().replace(u'\t','').replace(u'\n','')        
        s_52h = soup.find('table',{'class':'rwidth'}).find_all('tr')[1].find('td').find_all('em')[0].text
        s_52l = soup.find('table',{'class':'rwidth'}).find_all('tr')[1].find('td').find_all('em')[1].text
        s_target = soup.find('table',{'class':'rwidth'}).find_all('tr')[0].find('td').find_all('em')[1].text
        s_bae = soup.find('table',{'class':'per_table'}).find_all('tr')[3].find('td').find('em').text
        
        #print(s_name, s_price, s_ch, s_ch1, s_ch2)
        #r_data = [s_name, s_price, s_ch, s_ch1, s_ch2, s_pr1, s_pr2, s_pr3, s_pr4]
        r_data = [s_name, s_price, s_ch, s_ch1, s_ch2, s_pr3, s_pr4, s_siga, s_fore, s_52h, s_52l, s_target, s_bae]

        return r_data
    
    except:
        print('no data'+code)

In [13]:
df_inter = pd.read_csv('interest.csv', encoding="euc-kr")

codes = df_inter['종목코드']

#i = int()
list_realtime = []

for code in codes:
    #print(code[2:])
    #i = i + 1
    #print(i)
    data1 = naver_data(code[2:])
    #print(data1)
    try:
        list_realtime.append(
            {
            '종목명'  :data1[0].replace('\n',''),
            '현재가'  :data1[1],
            '변동금액':int(data1[2].replace(',','')),
            '손익'    :data1[3],
            '비율'    :float(data1[4]),
            '고가'    :data1[5],
            '저가'    :data1[6],
            '시총'    :data1[7],
            '외국인'    :data1[8],
            '52H'    :data1[9],
            '52L'    :data1[10],
            '목표가'    :data1[11],
            '배당률'    :data1[12]                
            })
    
    except:
        print('no data1')

# display(list_realtime)
df_realtime = pd.DataFrame(list_realtime)
df_realtime_sort = df_realtime.sort_values(by=['손익','비율','변동금액'], ascending=False)

display(df_realtime_sort)    


no data090460
no data1
no data036930
no data1
no data086450
no data1


,종목명,현재가,변동금액,손익,비율,고가,저가,시총,외국인,52H,52L,목표가,배당률
16,LG전자우,"52,400",2100,-,3.85,"54,900","52,000","9,005",44.29%,"54,900","31,900","110,850",2.00
21,이오테크닉스,"277,500",7500,-,2.63,"284,000","274,000","3조4,187",22.56%,"301,500","111,200","325,750",0.18
7,LG전자,"97,100",2600,-,2.61,"99,800","96,100","15조8,163",32.25%,"102,500","64,100","110,850",1.03
17,LX세미콘,"52,500",1000,-,1.87,"53,400","52,400","8,539",28.84%,"68,000","46,150",N/A,4.57
26,심텍,"56,500",1000,-,1.74,"57,200","55,500",2조877,11.21%,"67,100","9,690","66,000",0.18
35,에코마케팅,"10,620",140,-,1.30,"10,810","10,490","3,292",17.13%,"15,490","7,390","17,500",4.71
8,SK텔레콤,"53,900",500,-,0.92,"54,400","53,700","11조5,772",73.93%,"59,100","50,400","62,528",6.57
18,케이씨텍,"38,600",250,-,0.64,"38,950","38,000","7,986",10.93%,"46,150","22,700","47,500",0.70
34,솔트룩스,"31,150",200,-,0.64,"31,600","30,800","3,923",4.03%,"58,900","20,300",N/A,N/A
6,삼성전기,"262,000",1000,-,0.38,"267,500","257,500","19조5,697",38.12%,"267,500","105,600","298,238",0.69
